# Web Crawler 

## 安裝環境

In [1]:
# pip install jupyter notebook

# 對網路發動請求的套件，可實作對網頁做get、post等HTTP協定的行為
# pip install requests

# 借助網頁的結構特性來解析網頁的工具，只需要簡單的幾條指令就可以提取HTML標籤裡的元素
# pip install beautifulsoup4


## 實作1: 爬取PTT八卦版的文章資訊

### 以PTT八卦版為例 : https://www.ptt.cc/bbs/Gossiping/index.html

### Step1. import 套件

In [2]:
import requests
from bs4 import BeautifulSoup

# 把Warning訊息去掉
requests.packages.urllib3.disable_warnings()

***

### Step2. 將網頁Get下來

In [3]:
# 設定proxy 在公司內需使用
http_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
https_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
ftp_proxy = "http://auhqproxy.corpnet.auo.com:8080/"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }
            
#r = requests.get(url, proxies=proxyDict, verify=False)

In [4]:
# 要爬的網站
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

***

### Step3. 處理PTT網站詢問是否已滿18歲的問題
1. 進入網站後會看到讓使用者點選「是否已滿18歲」按鈕

2. 可開啟瀏覽器開發者模式(F12)，並點選至network(網路)，觀察點選「已滿18歲」後，會送給伺服器之封包內容。

3. 發現封包以POST發送，並且封包內容有cookie和參數

4. 我們的程式碼可以透過POST的方式，送出cookie和特定的參數

5. 可以藉由requests的一個參數來儲存自身的cookie
   Session會將你送出的requests所收到的cookies全部儲存起來並且在發送下一次請求時送出對應的參數。

In [5]:
# 處理是否已滿18歲的問題
payload = {
    "from":"/bbs/Gossiping/index.html",
    "yes":"yes"
}

rs = requests.session()

# 送出資訊已滿18歲的資訊
# verify=False 是為了避免SSL認證
res = rs.post("https://www.ptt.cc/ask/over18", proxies=proxyDict, verify=False, data=payload)

# 將此頁面的HTML GET下來
res = rs.get(url, proxies=proxyDict, verify=False)

# 印出HTML
print(res.text)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/co

***

### Step4. 以HTML標籤及屬性搜尋節點

### soup物件包含了整個網頁的HTML程式碼，接下來利用BeautifulSoup套件進行節點的搜尋。

In [6]:
# 將網頁資料以html.parser
soup = BeautifulSoup(res.text, "html.parser")

### 1. find()
    - 只搜尋第一個符合條件的HTML節點，傳入要搜尋的標籤名稱

In [7]:
result = soup.find("div")
print(result)

<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>


### 2. find_all()
    - 搜尋所有符合條件的HTML節點，傳入要搜尋的標籤名稱
    - 可以利用關鍵字參數(Keyword Argument)指定其屬性值。
    - 由於執行結果可能會搜出許多的HTML內容，所以最後也可以利用limit關鍵字參數(Keyword Argument)限制搜尋的節點數量

In [8]:
result = soup.find_all("div", id="action-bar-container", limit=3)
print(result)

[<div id="action-bar-container">
<div class="action-bar">
<div class="btn-group btn-group-dir">
<a class="btn selected" href="/bbs/Gossiping/index.html">看板</a>
<a class="btn" href="/man/Gossiping/index.html">精華區</a>
</div>
<div class="btn-group btn-group-paging">
<a class="btn wide" href="/bbs/Gossiping/index1.html">最舊</a>
<a class="btn wide" href="/bbs/Gossiping/index39068.html">‹ 上頁</a>
<a class="btn wide disabled">下頁 ›</a>
<a class="btn wide" href="/bbs/Gossiping/index.html">最新</a>
</div>
</div>
</div>]


### 2.1 同時搜尋多個HTML標籤

In [9]:
result = soup.find_all(["div", "a"], limit=5)
print(result)

[<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>, <div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>, <a href="/bbs/" id="logo">批踢踢實業坊</a>, <a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>, <a class="right small" href="/about.html">關於我們</a>]


### 3. 搜尋子節點
### select_one()
    - 當某一節點下只有單個子節點時，可以利用select_one()
### select()
    - 當某一節點下有多個子節點時，可以利用select()

In [10]:
result = soup.find("div")

In [11]:
print(result)

<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>


In [12]:
print(result.select_one("a"))

<a href="/bbs/" id="logo">批踢踢實業坊</a>


In [13]:
print(result.select("a"))

[<a href="/bbs/" id="logo">批踢踢實業坊</a>, <a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>, <a class="right small" href="/about.html">關於我們</a>, <a class="right small" href="/contact.html">聯絡資訊</a>]


***

### Step 5. 以CSS屬性搜尋節點
    - 依據HTML的css屬性來進行節點的搜尋，需使用class_ 進行css屬性值的指定

### 1. find()
    - 搜尋第一個符合指定的HTML標籤及css屬性值的節點

In [14]:
result = soup.find("div", class_="r-ent")
print(result)

<div class="r-ent">
<div class="nrec"><span class="hl f2">3</span></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650356571.A.BBE.html">[問卦] 今天咖哩神是不是打了48分鐘？</a>
</div>
<div class="meta">
<div class="author">macbook12</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6%5D+%E4%BB%8A%E5%A4%A9%E5%92%96%E5%93%A9%E7%A5%9E%E6%98%AF%E4%B8%8D%E6%98%AF%E6%89%93%E4%BA%8648%E5%88%86%E9%90%98%EF%BC%9F">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3Amacbook12">搜尋看板內 macbook12 的文章</a></div>
</div>
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>
</div>


### 2. find_all()
    - 搜尋網頁中符合指定的HTML標籤及css屬性值的所有節點

In [15]:
result = soup.find_all("div", class_="r-ent", limit=3)
print(result)

[<div class="r-ent">
<div class="nrec"><span class="hl f2">3</span></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650356571.A.BBE.html">[問卦] 今天咖哩神是不是打了48分鐘？</a>
</div>
<div class="meta">
<div class="author">macbook12</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6%5D+%E4%BB%8A%E5%A4%A9%E5%92%96%E5%93%A9%E7%A5%9E%E6%98%AF%E4%B8%8D%E6%98%AF%E6%89%93%E4%BA%8648%E5%88%86%E9%90%98%EF%BC%9F">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3Amacbook12">搜尋看板內 macbook12 的文章</a></div>
</div>
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>
</div>, <div class="r-ent">
<div class="nrec"><span class="hl f2">2</span></div>
<div class="title">
			
				(本文已被刪除) [samp05]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>
</div>

### 3. select()
    - 單純只想要透過css屬性值來進行HTML節點的搜尋

In [16]:
result = soup.select(".r-ent", limit=3)
print(result)

[<div class="r-ent">
<div class="nrec"><span class="hl f2">3</span></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650356571.A.BBE.html">[問卦] 今天咖哩神是不是打了48分鐘？</a>
</div>
<div class="meta">
<div class="author">macbook12</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6%5D+%E4%BB%8A%E5%A4%A9%E5%92%96%E5%93%A9%E7%A5%9E%E6%98%AF%E4%B8%8D%E6%98%AF%E6%89%93%E4%BA%8648%E5%88%86%E9%90%98%EF%BC%9F">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3Amacbook12">搜尋看板內 macbook12 的文章</a></div>
</div>
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>
</div>, <div class="r-ent">
<div class="nrec"><span class="hl f2">2</span></div>
<div class="title">
			
				(本文已被刪除) [samp05]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>
</div>

***

### Step 6. 搜尋父節點
    - 以上皆為向下的搜尋節點方式，如果想要從某一個節點向上搜尋，可以透過find_parent()跟find_parents()

In [17]:
result = soup.find("div", class_="title")
parents = result.find_parents("div")
print(parents)

[<div class="r-ent">
<div class="nrec"><span class="hl f2">3</span></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650356571.A.BBE.html">[問卦] 今天咖哩神是不是打了48分鐘？</a>
</div>
<div class="meta">
<div class="author">macbook12</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6%5D+%E4%BB%8A%E5%A4%A9%E5%92%96%E5%93%A9%E7%A5%9E%E6%98%AF%E4%B8%8D%E6%98%AF%E6%89%93%E4%BA%8648%E5%88%86%E9%90%98%EF%BC%9F">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3Amacbook12">搜尋看板內 macbook12 的文章</a></div>
</div>
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>
</div>, <div class="r-list-container action-bar-margin bbs-screen">
<div class="search-bar">
<form action="search" id="search-bar" type="get">
<input class="query" name="q" placeholder="搜尋文章⋯" type="text" value=""/>
</form>
</div>
<div class="r-ent">
<div class="nrec"><span class="hl f2">3

***

### Step 7. 搜尋前、後節點
    - 在同一層級的節點，想要搜尋前一個節點，可以透過find_previous_siblings()
    - 在同一層級的節點，想要搜尋後一個節點，可以透過find_next_siblings()

In [20]:
#print(res.text)

In [19]:
result = soup.find("div", class_="title")
previous_node = result.find_previous_siblings("div")
print(previous_node)

[<div class="nrec"><span class="hl f2">3</span></div>]


In [21]:
result = soup.find("div", class_="title")
previous_node = result.find_next_siblings("div")
print(previous_node)

[<div class="meta">
<div class="author">macbook12</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E5%95%8F%E5%8D%A6%5D+%E4%BB%8A%E5%A4%A9%E5%92%96%E5%93%A9%E7%A5%9E%E6%98%AF%E4%B8%8D%E6%98%AF%E6%89%93%E4%BA%8648%E5%88%86%E9%90%98%EF%BC%9F">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3Amacbook12">搜尋看板內 macbook12 的文章</a></div>
</div>
</div>
<div class="date"> 4/19</div>
<div class="mark"></div>
</div>]


***

### Step 8. 取得屬性值
    - 如果想要取得某一個節點中的屬性值，則可以利用BeautifulSoup套件的get()

In [22]:
result = soup.find_all("div", class_="r-ent")
for title in result:
    print(title.select_one("a"))

<a href="/bbs/Gossiping/M.1650356571.A.BBE.html">[問卦] 今天咖哩神是不是打了48分鐘？</a>
None
<a href="/bbs/Gossiping/M.1650356650.A.FEA.html">[新聞] 烏克蘭平民遭屠殺 馬克宏：已停止和蒲亭</a>
<a href="/bbs/Gossiping/M.1650356664.A.488.html">[問卦] 有人覺得兩歲小童很可憐嗎？</a>
<a href="/bbs/Gossiping/M.1650356717.A.A00.html">[問卦] 黃金周要去哪的八卦?</a>
<a href="/bbs/Gossiping/M.1650356731.A.49E.html">[問卦] 總統會有職業倦怠嗎？</a>
<a href="/bbs/Gossiping/M.1650356783.A.D8C.html">[問卦] 現在還是教I am fine thank you 嗎</a>
<a href="/bbs/Gossiping/M.1650356806.A.380.html">Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：</a>
<a href="/bbs/Gossiping/M.1650356818.A.665.html">[問卦] 扣子一定要扣好484一種強迫症？</a>
<a href="/bbs/Gossiping/M.1650356856.A.71A.html">Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：</a>
<a href="/bbs/Gossiping/M.1650356885.A.EA4.html">[問卦] 確診屁孩去買禮物打籃球算自主應變嗎？</a>
<a href="/bbs/Gossiping/M.1650356902.A.4FE.html">[問卦] 什麼是自己可以別人不行的舉例</a>
<a href="/bbs/Gossiping/M.1650356903.A.673.html">[問卦] 確診人數 vs 確診比例</a>
<a href="/bbs/Gossiping/M.1650356909.A.E91.html">Re: [問卦] 我們比其他國家晚了一年爆發為何要民眾</a>
<a href

In [35]:
result = soup.find_all("div", class_="r-ent")
for title in result:
    if title.select_one("a") != None:
        print(title.select_one("a").get("href"), " : ", title.select_one("a").text)
        #print(title.select_one("a").get("href"), " : ", title.select_one("a").getText())

/bbs/Gossiping/M.1650356571.A.BBE.html  :  [問卦] 今天咖哩神是不是打了48分鐘？
/bbs/Gossiping/M.1650356650.A.FEA.html  :  [新聞] 烏克蘭平民遭屠殺 馬克宏：已停止和蒲亭
/bbs/Gossiping/M.1650356664.A.488.html  :  [問卦] 有人覺得兩歲小童很可憐嗎？
/bbs/Gossiping/M.1650356717.A.A00.html  :  [問卦] 黃金周要去哪的八卦?
/bbs/Gossiping/M.1650356731.A.49E.html  :  [問卦] 總統會有職業倦怠嗎？
/bbs/Gossiping/M.1650356783.A.D8C.html  :  [問卦] 現在還是教I am fine thank you 嗎
/bbs/Gossiping/M.1650356806.A.380.html  :  Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：
/bbs/Gossiping/M.1650356818.A.665.html  :  [問卦] 扣子一定要扣好484一種強迫症？
/bbs/Gossiping/M.1650356856.A.71A.html  :  Re: [新聞] 確診暴增「衛生局來不及通知」 陳時中：
/bbs/Gossiping/M.1650356885.A.EA4.html  :  [問卦] 確診屁孩去買禮物打籃球算自主應變嗎？
/bbs/Gossiping/M.1650356902.A.4FE.html  :  [問卦] 什麼是自己可以別人不行的舉例
/bbs/Gossiping/M.1650356903.A.673.html  :  [問卦] 確診人數 vs 確診比例
/bbs/Gossiping/M.1650356909.A.E91.html  :  Re: [問卦] 我們比其他國家晚了一年爆發為何要民眾
/bbs/Gossiping/M.1650356981.A.51E.html  :  Re: [問卦] Google達美樂,第一個廣告跳必勝客？！
/bbs/Gossiping/M.1650357003.A.BB7.html  :  [問卦] 自主應變 = 失控了 大家自求多福？
/